In [2]:
import ultralytics
from ultralytics import YOLO
import math

In [ ]:
from sort import *

In [ ]:
import cv2
import cvzone
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [ ]:
import numpy as np
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,BatchNormalization,Dropout
from keras import Sequential
from twilio.rest import Client
import tensorflow_datasets as tfds

In [ ]:
CNNmodel=Sequential()
CNNmodel.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(28,28,1)))
CNNmodel.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
CNNmodel.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
CNNmodel.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))

CNNmodel.add(Flatten())
CNNmodel.add(Dense(128,activation='relu'))
CNNmodel.add(Dense(128,activation='relu'))
CNNmodel.add(Dense(128,activation='relu'))


In [ ]:
speed_limit=200
speed=0
start_time=0
end_time=0

cap=cv2.VideoCapture('C:/Users/HP/Downloads/2165-155327596_small.mp4')
model=YOLO('../YOLO-Weight/yolov8n.pt')
#numbers_of_vehicals=0

classNames = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", 
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", 
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", 
    "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", 
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", 
    "bed", "dining table", "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone", 
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", 
    "teddy bear", "hair dryer", "toothbrush"
]

In [ ]:
mask=cv2.imread('C:/Users/HP/Downloads/mask.png')

tracker=Sort(max_age=20,min_hits=3,iou_threshold=0.3)



totalCount=[]



frame_width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("frame_width",frame_width)
print("frame_height",frame_height)

mask=cv2.resize(mask,(frame_width,frame_height))


In [ ]:
try:
    while True:
        success, img = cap.read()
        
        if not success:
            print("Failed to read frame. Exiting...")
            break
        
        imgRegion = cv2.bitwise_and(img, mask)
        results = model(imgRegion, stream=True)
        detections = np.empty((0, 5))
        
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1
                
                conf = math.ceil((box.conf[0] * 100)) / 100
                line = cv2.line(img, (0, int(frame_height / 2) - 40), (frame_width, int(frame_height / 2) - 40), (200, 100, 255), 5)
                line2 = cv2.line(img, (0, int(frame_height / 2) + 35), (frame_width, int(frame_height / 2) + 35), (0, 0, 0), 5)
                 
                cls = classNames[int(box.cls[0])]
                if cls in ['car', 'bicycle', 'motorbike', 'truck', 'bus'] and conf > 0.3:
                    currentArray = np.array([x1, y1, x2, y2, conf])
                    detections = np.vstack((detections, currentArray))

        resultsTracker = tracker.update(detections)
        for result in resultsTracker:
            x1, y1, x2, y2, id = result
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            
            cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=4, colorR=(255, 0, 255))
            cvzone.putTextRect(img, f'{int(id)}', (max(0, x1), max(35, y1)), scale=0.7, thickness=1)
            cx, cy = x1 + w // 2, y1 + h // 2
            cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            if 0 < cx < frame_width and int(frame_height / 2) - 40 - 20 < cy < int(frame_height / 2) - 40 + 20:
                if totalCount.count(id) == 0:
                    totalCount.append(int(id))
                    x = len(totalCount)
                    line = cv2.line(img, (0, int(frame_height / 2) - 40), (frame_width, int(frame_height / 2) - 40), (0, 255, 0), 5)

            if 0 < cx < frame_width and int(frame_height / 2) - 60 < cy < int(frame_height / 2) - 20:
                start_time = time.time()
            if 0 < cx < frame_width and int(frame_height / 2) + 35 - 20 < cy < int(frame_height / 2) + 35 + 20:
                end_time = time.time()
            if start_time != 0 and end_time != 0 and start_time < end_time:
                speed = 45 / (end_time - start_time) + 200
                print(f'Speed: {abs(speed):.2f} km/h')

            if speed > speed_limit:
                cropped_vehicle = img[y1:y2, x1:x2]
                filename = f"C:/Users/HP/Desktop/jupyterNotebook/projects/photos_of_overspeedCars{int(time.time())}.jpg"
                
                cv2.imwrite(filename, cropped_vehicle)
                print(f"Saved image: {filename}")
            else:
                print("Speed calculation skipped due to invalid time interval.")

        cvzone.putTextRect(img, f'Count: {x}', (50, 50))
        cv2.imshow("img", img)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting...")
            break
except KeyboardInterrupt:
    print("Terminated by user.")
finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Resources released.")


In [ ]:
account_sid = 'ACe8722b25a1c1b7a58bfdc3e1124d43c9'
auth_token = '42bb0aec5fd958190de710444e1a9696'
client = Client(account_sid, auth_token)

# Function to map the predicted class index to a character
def class_to_char(predicted_class):
    if predicted_class < 10:
        return chr(predicted_class + 48)  # Digits 0-9
    elif predicted_class < 36:
        return chr(predicted_class - 10 + 65)  # Uppercase A-Z
    else:
        return chr(predicted_class - 36 + 97)  # Lowercase a-z

# Preprocess the dataset (normalize images)
def preprocess(example):
    image = tf.cast(example['image'], tf.float32) / 255.0
    label = example['label']
    return image, label

# Load and preprocess the EMNIST dataset
emnist_builder = tfds.builder('emnist/byclass')
emnist_builder.download_and_prepare()
emnist = emnist_builder.as_dataset(split='train')
train_dataset = emnist.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(62, activation='softmax')  # 62 classes for digits and letters
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=50)


In [ ]:
def processing_image(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
    img_resized = cv2.resize(img_gray, (28, 28))
    img_array = img_resized / 255.0
    return img_array.reshape(1, 28, 28, 1)

# Function to segment characters from the image
def segment_characters(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
    char_bboxes = [cv2.boundingRect(c) for c in contours if cv2.contourArea(c) > 100]
    return char_bboxes, binary

# Folder path containing the images
path_of_folder = 'C:/Users/HP/Desktop/jupyterNotebook/projects/photos_of_overspeedCars'
images = os.listdir(path_of_folder)

In [ ]:
for img_name in images:
    img_path = os.path.join(path_of_folder, img_name)
    
    photo = None
    while photo is None:
        photo = cv2.imread(img_path)
        if photo is None:
            print(f"Retrying: Unable to load image {img_path}")

    if photo is None:
        print(f"Error: Unable to load image {img_path}")
        continue

    # Segment characters
    char_bboxes, binary = segment_characters(photo)
    filtered_bboxes = [bbox for bbox in char_bboxes if bbox[2] > 5 and bbox[3] > 5]
    l = []

    # Predict characters
    for bbox in filtered_bboxes:
        x, y, w, h = bbox
        char_img = binary[y:y + h, x:x + w]
        input_img = processing_image(char_img)
        prediction = model.predict(input_img)
        predicted_class = np.argmax(prediction)
        predicted_char = class_to_char(predicted_class)
        l.append(predicted_char)

        # Draw bounding box and text
        cv2.rectangle(photo, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cvzone.putTextRect(photo, predicted_char, (x, y + 30), scale=1, thickness=1, colorR=(0, 0, 255))

    # Combine characters into a plate number
    plate_number = ''.join(l)
    print(f"Detected Plate Number: {plate_number}")

    # Send message
    try:
        message = client.messages.create(
            body=f"The car with number plate {plate_number} has over-speeded.",
            from_='+12314488432',
            to='+919359645981'
        )
        print(f"Message sent with SID: {message.sid}")
    except Exception as e:
        print(f"Failed to send message: {e}")

    # Display the processed image
    cv2.imshow("Segmented Characters", photo)
    cv2.waitKey(0)

cv2.destroyAllWindows()